# Chapter 61: Mobile App CI/CD

Mobile application deployment fundamentally differs from server-side CI/CD in one critical aspect: the target platform is not infrastructure you control, but tightly regulated ecosystems governed by Apple and Google. Unlike deploying containers to Kubernetes where you own the runtime environment, mobile CI/CD must navigate code signing requirements, provisioning profiles, app store review processes, and binary distribution models that introduce latency and gatekeeping foreign to backend deployment. Additionally, mobile apps execute on heterogeneous device fleets with varying screen sizes, OS versions, and hardware capabilities, requiring extensive testing across physical and virtual device matrices. This chapter examines how to automate iOS and Android builds, manage the complexity of code signing in ephemeral CI environments, implement cross-platform development pipelines for Flutter and React Native, and leverage over-the-air (OTA) update mechanisms to bypass app store delays for critical fixes.

## 61.1 Mobile Build Processes

Mobile build processes transform high-level source code into platform-specific binary artifacts—IPA files for iOS and APK/AAB (Android App Bundle) files for Android. Unlike container builds that layer filesystem changes, mobile builds compile native code, bundle assets, optimize resources, and cryptographically sign binaries to establish trust with operating system security models.

### Native vs. Cross-Platform Architectures

**Native Development:**
- **iOS**: Xcode build system compiling Swift/Objective-C, linking against iOS SDKs, producing ARM64 binaries. Requires macOS build agents due to Xcode licensing restrictions.
- **Android**: Gradle-based builds compiling Kotlin/Java, generating DEX bytecode, packaging resources. Builds on any JVM-capable OS.

**Cross-Platform:**
Frameworks like Flutter (Dart) and React Native (JavaScript) introduce an abstraction layer, compiling to native ARM code or embedding JavaScript engines. Their CI pipelines must handle dual-platform complexities while sharing business logic.

### Build Pipeline Stages

**Dependency Resolution:**
- **iOS**: CocoaPods, Swift Package Manager (SPM), or Carthage resolve frameworks. SPM integrates natively with Xcode; CocoaPods requires `pod install` execution.
- **Android**: Gradle downloads dependencies from Maven Central, Google, or private repositories. Dependency caching critical for build performance.

**Compilation:**
- **iOS**: Xcodebuild compiles Swift source, links frameworks, generates dSYM debug symbols.
- **Android**: Kotlin/Java compilation → DEX conversion → Resource merging → APK/AAB packaging.

**Asset Optimization:**
Image compression, font subsetting, and resource shrinking (Android's R8/ProGuard) reduce binary size. iOS uses Asset Catalogs for device-specific image variants (1x, 2x, 3x).

**Code Signing:**
Cryptographic signing proves app authenticity. iOS requires Apple-issued certificates and provisioning profiles; Android uses keystores generated by developers.

**Artifact Generation:**
- **iOS**: `.ipa` (iOS App Store Package) for distribution, `.app` for simulators.
- **Android**: `.apk` (Android Package) for direct installation, `.aab` (Android App Bundle) for Play Store submission (enables Dynamic Delivery).

## 61.2 iOS CI/CD

iOS CI/CD presents unique challenges: macOS build requirements, complex code signing infrastructure, and App Store Connect's asynchronous review process.

### Xcode Build System

Xcodebuild is the command-line interface to Apple's build system, enabling CI automation without GUI interaction:

```bash
# Build for simulator (testing)
xcodebuild \
  -workspace MyApp.xcworkspace \
  -scheme MyApp \
  -sdk iphonesimulator \
  -destination 'platform=iOS Simulator,name=iPhone 15,OS=17.0' \
  clean build

# Build for device (unsigned, testing)
xcodebuild \
  -workspace MyApp.xcworkspace \
  -scheme MyApp \
  -sdk iphoneos \
  -configuration Release \
  build

# Archive for distribution (signed)
xcodebuild \
  -workspace MyApp.xcworkspace \
  -scheme MyApp \
  -sdk iphoneos \
  -configuration Release \
  -archivePath $PWD/build/MyApp.xcarchive \
  archive
```

**XCConfig Files:**
Manage build settings externally for environment-specific configurations:

```bash
# Configs/Debug.xcconfig
BUNDLE_IDENTIFIER = com.company.myapp.debug
PRODUCT_NAME = MyApp Debug
CODE_SIGN_IDENTITY = iPhone Developer
PROVISIONING_PROFILE_SPECIFIER = match Development com.company.myapp.debug

# Configs/Release.xcconfig
BUNDLE_IDENTIFIER = com.company.myapp
PRODUCT_NAME = MyApp
CODE_SIGN_IDENTITY = iPhone Distribution
PROVISIONING_PROFILE_SPECIFIER = match AppStore com.company.myapp
```

### Code Signing and Certificates

iOS security requires three components:
1. **Signing Certificate**: Cryptographic identity (public/private key) issued by Apple.
2. **Provisioning Profile**: Links app ID, certificate, and device UDIDs (development) or distribution method (App Store, Enterprise, Ad Hoc).
3. **Bundle Identifier**: Unique app ID (e.g., `com.company.app`).

**Certificate Types:**
- **Development**: Signed apps run on registered devices only.
- **Distribution (App Store)**: For App Store submission; cannot run on devices directly.
- **Enterprise**: In-house distribution outside App Store (requires Apple Enterprise Program).

**Fastlane Match:**
Manage signing assets in Git repositories encrypted with GPG:

```ruby
# Gemfile
source "https://rubygems.org"
gem "fastlane"

# fastlane/Matchfile
git_url("https://github.com/company/ios-certificates.git")
storage_mode("git")
type("development") # or "appstore", "adhoc", "enterprise"
app_identifier(["com.company.app", "com.company.app.extension"])
username("dev@company.com")

# Fastfile lane
lane :beta do
  # Sync certificates/profiles
  match(type: "appstore", readonly: is_ci)
  
  # Build
  gym(
    workspace: "MyApp.xcworkspace",
    scheme: "MyApp",
    configuration: "Release",
    export_method: "app-store",
    output_directory: "build",
    xcargs: "-allowProvisioningUpdates"
  )
  
  # Upload to TestFlight
  pilot(
    skip_waiting_for_build_processing: true,
    changelog: "Bug fixes and performance improvements"
  )
end
```

**CI Implementation (GitHub Actions):**
```yaml
name: iOS Build and Deploy
on:
  push:
    branches: [main]

jobs:
  build:
    runs-on: macos-13  # macOS required for Xcode
    steps:
    - uses: actions/checkout@v3
    
    - name: Setup Ruby
      uses: ruby/setup-ruby@v1
      with:
        ruby-version: '3.2'
        bundler-cache: true
    
    - name: Setup Xcode
      uses: maxim-lobanov/setup-xcode@v1
      with:
        xcode-version: '15.0'
    
    - name: Cache Pods
      uses: actions/cache@v3
      with:
        path: Pods
        key: ${{ runner.os }}-pods-${{ hashFiles('**/Podfile.lock') }}
    
    - name: Install Dependencies
      run: |
        gem install bundler
        bundle install
        pod install
    
    - name: Setup Signing
      env:
        MATCH_PASSWORD: ${{ secrets.MATCH_PASSWORD }}
        MATCH_GIT_BASIC_AUTHORIZATION: ${{ secrets.GIT_PAT }}
      run: |
        bundle exec fastlane match appstore --readonly
    
    - name: Build and Upload
      env:
        APP_STORE_CONNECT_API_KEY: ${{ secrets.APP_STORE_CONNECT_API_KEY }}
      run: |
        bundle exec fastlane beta
```

### App Store Connect API

Modern iOS CI/CD uses App Store Connect API (JWT-based) rather than Apple IDs with 2FA:

```ruby
# Create API key in App Store Connect (Users and Access > Keys)
api_key = app_store_connect_api_key(
  key_id: "D383SF739",
  issuer_id: "69a6de8a-46c8-47e3-e053-5b8c7c11a4d1",
  key_content: ENV["APP_STORE_CONNECT_KEY"],  # PKCS#8 format
  duration: 1200,  # seconds
  in_house: false  # App Store vs Enterprise
)

upload_to_app_store(
  api_key: api_key,
  skip_screenshots: true,
  skip_metadata: true,
  submit_for_review: false,  # Manual review submission
  automatic_release: false   # Manual release after approval
)
```

## 61.3 Android CI/CD

Android CI/CD benefits from cross-platform build capability (Linux containers) and more flexible signing infrastructure, though Google Play's signing requirements have evolved with App Bundles.

### Gradle Build Configuration

**Build Types and Flavors:**
```groovy
// app/build.gradle
android {
    namespace 'com.company.app'
    compileSdk 34
    
    defaultConfig {
        applicationId "com.company.app"
        minSdk 24
        targetSdk 34
        versionCode System.getenv("CI_BUILD_NUMBER")?.toInteger() ?: 1
        versionName "1.0.0-${System.getenv('CI_COMMIT_SHA')?.take(7) ?: 'dev'}"
    }
    
    signingConfigs {
        release {
            storeFile file(System.getenv("KEYSTORE_PATH") ?: "debug.keystore")
            storePassword System.getenv("KEYSTORE_PASSWORD")
            keyAlias System.getenv("KEY_ALIAS")
            keyPassword System.getenv("KEY_PASSWORD")
        }
    }
    
    buildTypes {
        debug {
            applicationIdSuffix ".debug"
            debuggable true
        }
        release {
            minifyEnabled true
            proguardFiles getDefaultProguardFile('proguard-android-optimize.txt'), 'proguard-rules.pro'
            signingConfig signingConfigs.release
        }
    }
    
    flavorDimensions "environment"
    productFlavors {
        staging {
            dimension "environment"
            applicationIdSuffix ".staging"
            resValue "string", "app_name", "MyApp Staging"
            buildConfigField "String", "API_URL", "\"https://staging-api.company.com\""
        }
        production {
            dimension "environment"
            resValue "string", "app_name", "MyApp"
            buildConfigField "String", "API_URL", "\"https://api.company.com\""
        }
    }
}

// Version code management for AAB
android.applicationVariants.all { variant ->
    variant.outputs.all {
        outputFileName = "app-${variant.name}-${variant.versionName}.aab"
    }
}
```

**GitLab CI Android Pipeline:**
```yaml
stages:
  - build
  - test
  - deploy

variables:
  GRADLE_OPTS: "-Dorg.gradle.daemon=false -Dorg.gradle.configureondemand=true"
  KEYSTORE_PATH: "$CI_PROJECT_DIR/keystore.jks"

cache:
  key: ${CI_COMMIT_REF_SLUG}
  paths:
    - .gradle/
    - app/build/

before_script:
  - apt-get update -qq && apt-get install -y -qq openjdk-17-jdk
  - chmod +x ./gradlew

build:debug:
  stage: build
  script:
    - ./gradlew assembleDebug
  artifacts:
    paths:
      - app/build/outputs/apk/debug/
    expire_in: 1 week

build:release:
  stage: build
  script:
    # Decode keystore from base64 secret
    - echo "$KEYSTORE_FILE" | base64 -d > $KEYSTORE_PATH
    - ./gradlew bundleProductionRelease
  artifacts:
    paths:
      - app/build/outputs/bundle/productionRelease/
  only:
    - main

deploy:playstore:
  stage: deploy
  image: google/cloud-sdk:slim
  script:
    - echo "$PLAY_STORE_SERVICE_ACCOUNT" > service-account.json
    - gcloud auth activate-service-account --key-file service-account.json
    - |
      gcloud firebase appdistribution releases upload \
        app/build/outputs/bundle/productionRelease/app-production-release.aab \
        --app 1:1234567890:android:abcd1234 \
        --groups "qa-team"
  only:
    - main
```

### Android App Bundles (AAB)

AAB is now required for Play Store uploads. It enables Dynamic Delivery (serving only required resources for specific devices):

```yaml
# GitHub Actions for AAB upload
- name: Build AAB
  run: ./gradlew bundleRelease

- name: Sign AAB
  uses: r0adkll/sign-android-release@v1
  with:
    releaseDirectory: app/build/outputs/bundle/release
    signingKeyBase64: ${{ secrets.SIGNING_KEY }}
    alias: ${{ secrets.ALIAS }}
    keyStorePassword: ${{ secrets.KEY_STORE_PASSWORD }}
    keyPassword: ${{ secrets.KEY_PASSWORD }}

- name: Upload to Play Store
  uses: r0adkll/upload-google-play@v1
  with:
    serviceAccountJsonPlainText: ${{ secrets.SERVICE_ACCOUNT_JSON }}
    packageName: com.company.app
    releaseFiles: app/build/outputs/bundle/release/*.aab
    track: internal  # internal, alpha, beta, production
    whatsNewDirectory: distribution/whatsnew
    mappingFile: app/build/outputs/mapping/release/mapping.txt
```

## 61.4 Cross-Platform Frameworks

Flutter and React Native enable single-codebase deployment to both iOS and Android, but CI/CD pipelines must handle both platforms' specific requirements simultaneously.

### Flutter CI/CD

Flutter builds compile Dart code to native ARM libraries, embedding Flutter engine:

```yaml
# .github/workflows/flutter.yml
name: Flutter CI/CD
on:
  push:
    tags:
      - 'v*'

jobs:
  build:
    runs-on: macos-latest  # macOS for iOS builds
    steps:
    - uses: actions/checkout@v3
    
    - name: Setup Flutter
      uses: subosito/flutter-action@v2
      with:
        flutter-version: '3.16.0'
        channel: 'stable'
        cache: true
    
    - name: Install Dependencies
      run: flutter pub get
    
    - name: Analyze
      run: flutter analyze
    
    - name: Run Tests
      run: flutter test
    
    - name: Build iOS
      run: |
        flutter build ipa --release --export-options-plist=ios/ExportOptions.plist
      env:
        MATCH_PASSWORD: ${{ secrets.MATCH_PASSWORD }}
    
    - name: Build Android
      run: |
        flutter build appbundle --release
    
    - name: Upload Artifacts
      uses: actions/upload-artifact@v3
      with:
        name: release-builds
        path: |
          build/ios/ipa/*.ipa
          build/app/outputs/bundle/release/*.aab
```

**Codemagic (Flutter-specific CI):**
```yaml
# codemagic.yaml
workflows:
  release:
    name: Release Build
    environment:
      flutter: stable
      xcode: latest
      cocoapods: default
    triggering:
      events:
        - push
      branch_patterns:
        - pattern: main
          include: true
    scripts:
      - name: Get Flutter packages
        script: flutter packages pub get
      - name: iOS Build
        script: |
          flutter build ipa --release \
            --export-options-plist=/Users/builder/export_options.plist
        environment:
          APP_STORE_CONNECT_ISSUER_ID: 69a6de...
          APP_STORE_CONNECT_KEY_IDENTIFIER: D383SF739
          APP_STORE_CONNECT_PRIVATE_KEY: ${APP_STORE_CONNECT_KEY}
      - name: Android Build
        script: |
          flutter build appbundle --release \
            --build-number=$(($PROJECT_BUILD_NUMBER)) \
            --build-name=1.0.0
    artifacts:
      - build/ios/ipa/*.ipa
      - build/app/outputs/bundle/release/*.aab
    publishing:
      app_store_connect:
        api_key: ${APP_STORE_CONNECT_KEY}
        submit_to_testflight: true
      google_play:
        credentials: ${GOOGLE_PLAY_SERVICE_ACCOUNT}
        track: internal
```

### React Native CI/CD

React Native requires both Node.js and native toolchain setup:

```yaml
name: React Native Build
on:
  push:
    branches: [main]

jobs:
  build-ios:
    runs-on: macos-latest
    steps:
    - uses: actions/checkout@v3
    
    - name: Setup Node
      uses: actions/setup-node@v3
      with:
        node-version: '18'
        cache: 'npm'
    
    - name: Install Dependencies
      run: |
        npm ci
        cd ios && pod install && cd ..
    
    - name: Build iOS
      run: |
        xcodebuild \
          -workspace ios/MyApp.xcworkspace \
          -scheme MyApp \
          -configuration Release \
          -sdk iphoneos \
          -archivePath $PWD/build/MyApp.xcarchive \
          archive
    
    - name: Export IPA
      run: |
        xcodebuild \
          -exportArchive \
          -archivePath $PWD/build/MyApp.xcarchive \
          -exportOptionsPlist ios/ExportOptions.plist \
          -exportPath $PWD/build

  build-android:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    
    - name: Setup Node
      uses: actions/setup-node@v3
      with:
        node-version: '18'
        cache: 'npm'
    
    - name: Setup JDK
      uses: actions/setup-java@v3
      with:
        java-version: '17'
        distribution: 'temurin'
    
    - name: Install Dependencies
      run: npm ci
    
    - name: Build Android
      run: |
        cd android
        ./gradlew assembleRelease
```

## 61.5 App Store Deployment

App store deployment involves more than binary upload—metadata, screenshots, and review compliance must be automated.

### Fastlane Deliver

Automate App Store metadata and screenshot submission:

```ruby
# fastlane/Deliverfile
username("dev@company.com")
app_identifier("com.company.app")
ipa("./build/MyApp.ipa")
submit_for_review(true)
automatic_release(false)
force(true)  # Skip HTML report verification in CI

# Metadata in fastlane/metadata/en-US/
# - description.txt
# - keywords.txt
# - release_notes.txt
# - support_url.txt
# - marketing_url.txt

# Screenshots configuration
lane :screenshots do
  capture_screenshots(
    workspace: "MyApp.xcworkspace",
    scheme: "MyAppUITests",
    devices: ["iPhone 15 Pro", "iPhone 15 Pro Max", "iPad Pro (12.9-inch)"],
    languages: ["en-US", "de-DE", "fr-FR"],
    output_directory: "./fastlane/screenshots"
  )
  
  frame_screenshots(
    white: true,
    path: "./fastlane/screenshots"
  )
end

lane :release do
  screenshots if is_ci == false  # Generate locally, reuse in CI
  
  deliver(
    submit_for_review: true,
    submission_information: {
      add_id_info_serves_ads: false,
      add_id_info_uses_idfa: false,
      add_id_info_limits_tracking: true,
      export_compliance_platform: 'ios',
      export_compliance_compliance_required: false,
      export_compliance_encryption_updated: false,
      export_compliance_app_type: nil,
      export_compliance_uses_encryption: false,
      export_compliance_is_exempt: false,
      export_compliance_contains_third_party_cryptography: false,
      export_compliance_contains_proprietary_cryptography: false,
      export_compliance_available_on_french_store: true
    }
  )
end
```

### Google Play Deployment

```ruby
lane :deploy_play_store do
  upload_to_play_store(
    track: 'internal',  # internal, alpha, beta, production
    aab: 'build/app-production-release.aab',
    skip_upload_metadata: false,
    skip_upload_images: false,
    skip_upload_screenshots: false,
    release_status: 'draft',  # completed, draft, halted, inProgress
    metadata_path: './fastlane/metadata/android'
  )
end
```

### Huawei AppGallery (China Distribution)

For markets without Google Play:

```ruby
lane :huawei do
  huawei_appgallery_connect(
    client_id: ENV["HUAWEI_CLIENT_ID"],
    client_secret: ENV["HUAWEI_CLIENT_SECRET"],
    app_id: "123456789",
    apk_path: "./build/app.apk",
    submit_for_review: false
  )
end
```

## 61.6 Testing Mobile Apps

Mobile testing requires validation across device matrices, OS versions, and hardware capabilities (camera, GPS, sensors).

### Unit and Widget Tests

**Flutter:**
```bash
flutter test --coverage
flutter test integration_test/app_test.dart
```

**React Native (Jest + React Native Testing Library):**
```json
{
  "scripts": {
    "test": "jest --coverage",
    "test:e2e": "detox test --configuration ios.sim.release"
  }
}
```

### UI Automation with Appium

Appium drives native apps using WebDriver protocol:

```javascript
// wdio.conf.js (WebDriverIO)
exports.config = {
  runner: 'local',
  port: 4723,
  specs: ['./test/specs/**/*.js'],
  capabilities: [{
    platformName: 'iOS',
    'appium:platformVersion': '17.0',
    'appium:deviceName': 'iPhone 15',
    'appium:automationName': 'XCUITest',
    'appium:app': './build/MyApp.app',
    'appium:noReset': true
  }, {
    platformName: 'Android',
    'appium:platformVersion': '14.0',
    'appium:deviceName': 'Pixel 7',
    'appium:automationName': 'UiAutomator2',
    'appium:app': './build/app.apk',
    'appium:appPackage': 'com.company.app',
    'appium:appActivity': '.MainActivity'
  }],
  framework: 'mocha',
  reporters: ['spec', ['allure', { outputDir: 'allure-results' }]]
};

// test/specs/login.e2e.js
describe('Login Flow', () => {
  it('should login with valid credentials', async () => {
    await $('~email-input').setValue('user@example.com');
    await $('~password-input').setValue('password123');
    await $('~login-button').click();
    
    await expect($('~home-screen')).toBeDisplayed();
  });
});
```

**CI Integration:**
```yaml
- name: Start Appium Server
  run: |
    npm install -g appium
    appium &

- name: Run E2E Tests
  run: |
    npm run test:e2e:ci
```

### Cloud Device Farms

**Firebase Test Lab:**
```yaml
- name: Run Instrumentation Tests
  run: |
    gcloud firebase test android run \
      --type instrumentation \
      --app app/build/outputs/apk/debug/app-debug.apk \
      --test app/build/outputs/apk/androidTest/debug/app-debug-androidTest.apk \
      --device model=redfin,version=30,locale=en,orientation=portrait \
      --device model=griffin,version=24,locale=de,orientation=landscape \
      --timeout 30m \
      --results-bucket gs://test-lab-results
```

**AWS Device Farm:**
```yaml
- name: AWS Device Farm Test
  run: |
    aws devicefarm create-upload \
      --project-arn $DEVICE_FARM_PROJECT_ARN \
      --name app-debug.apk \
      --type ANDROID_APP
    
    # Upload APK and test package, schedule run
```

**BrowserStack:**
```ruby
lane :browserstack do
  upload_to_browserstack_app_automate(
    browserstack_username: ENV["BROWSERSTACK_USERNAME"],
    browserstack_access_key: ENV["BROWSERSTACK_ACCESS_KEY"],
    file_path: "build/app.ipa"
  )
  
  # Trigger automated tests via API
  sh("curl -X POST https://api.browserstack.com/app-automate/espresso/v2/build ...")
end
```

## 61.7 Code Signing and Security

Managing signing credentials in CI requires balancing security with automation.

### iOS Certificate Management Strategies

**Strategy 1: Fastlane Match (Recommended)**
Encrypted Git repository storing certificates and profiles:
```bash
# Setup (one time)
fastlane match init
fastlane match appstore

# CI Usage (read-only)
fastlane match appstore --readonly
```

**Strategy 2: GitHub Secrets**
Store Base64-encoded certificates:
```yaml
- name: Install Certificates
  run: |
    echo "$DEVELOPER_CERTIFICATE" | base64 -d > certificate.p12
    security create-keychain -p "" build.keychain
    security import certificate.p12 -k build.keychain -P "$CERTIFICATE_PASSWORD" -T /usr/bin/codesign
    security set-keychain-settings -t 3600 -u build.keychain
    security unlock-keychain -p "" build.keychain
```

### Android Keystore Management

**Google Play App Signing:**
Google manages signing key; you upload signing key (for updates) and upload key (for new uploads):

```yaml
- name: Sign AAB
  run: |
    echo "$UPLOAD_KEYSTORE" | base64 -d > upload.keystore
    jarsigner -keystore upload.keystore \
      -storepass $STORE_PASSWORD \
      -keypass $KEY_PASSWORD \
      app.aab $KEY_ALIAS
```

**Environment Security:**
Never commit keystores. Use CI secret managers:
- GitHub: Repository secrets + Environment protection rules
- GitLab: CI/CD variables (masked, protected)
- Azure DevOps: Variable groups with security restrictions

## 61.8 Over-the-Air (OTA) Updates and PWAs

When app store review delays (24-48 hours) block critical fixes, OTA mechanisms enable immediate JavaScript/asset updates without binary changes.

### React Native CodePush

Microsoft CodePush (App Center) hosts JavaScript bundles separately from native binaries:

```javascript
// App.js
import codePush from 'react-native-code-push';

const App = () => {
  // App component
};

// Check for updates on app resume
export default codePush({
  checkFrequency: codePush.CheckFrequency.ON_APP_RESUME,
  installMode: codePush.InstallMode.ON_NEXT_RESTART,
  mandatoryInstallMode: codePush.InstallMode.IMMEDIATE
})(App);
```

**CI/CD Integration:**
```yaml
- name: Deploy CodePush Update
  run: |
    npm install -g appcenter-cli
    appcenter codepush release-react \
      -a MyOrg/MyApp \
      -d Production \
      -m true \
      --description "Critical bug fix"
```

**Limitations:**
- Cannot update native code (Android/iOS native modules)
- Apple requires that updates don't significantly change app purpose
- Must maintain backward compatibility with native modules

### Expo Updates (EAS)

For Expo managed workflow:

```yaml
- name: Publish Expo Update
  run: |
    npx eas update --channel production --message "Bug fixes"
```

### Progressive Web Apps (PWA)

PWAs bypass app stores entirely, offering mobile-optimized web experiences with offline capability:

**Deployment Pipeline:**
```yaml
name: PWA Deploy
on:
  push:
    branches: [main]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    
    - name: Build PWA
      run: |
        npm ci
        npm run build
    
    - name: Deploy to Firebase Hosting
      uses: FirebaseExtended/action-hosting-deploy@v0
      with:
        repoToken: '${{ secrets.GITHUB_TOKEN }}'
        firebaseServiceAccount: '${{ secrets.FIREBASE_SERVICE_ACCOUNT }}'
        channelId: live
    
    - name: Update Service Worker
      run: |
        # Ensure cache-busting for new deployments
        workbox generateSW workbox-config.js
```

**Capacitor/Ionic Hybrid:**
Wrap PWA as native app when store distribution required:

```bash
npx cap sync ios
npx cap open ios
# Then use standard iOS CI/CD pipeline
```

---

## Chapter Summary and Preview

This chapter explored the unique constraints of mobile CI/CD, where deployment targets are tightly controlled ecosystems rather than self-managed infrastructure. We established that iOS CI/CD requires macOS build agents and complex code signing infrastructure managed through Fastlane Match or secure secret storage, while Android CI/CD benefits from Linux-based builds but must navigate App Bundle requirements and Play Store signing policies.

Cross-platform frameworks (Flutter and React Native) introduce efficiency through shared codebases but compound CI complexity by requiring both platform toolchains simultaneously. App store deployment automation extends beyond binary upload to include metadata management, screenshot generation, and compliance with review guidelines that vary by platform and jurisdiction.

Testing strategies for mobile apps must account for device fragmentation through cloud-based device farms (Firebase Test Lab, AWS Device Farm, BrowserStack) and UI automation frameworks like Appium. Security considerations emphasized the management of signing certificates in ephemeral CI environments, with Fastlane Match providing a Git-based encrypted solution for iOS and environment variables securing Android keystores.

Over-the-Air update mechanisms (CodePush, Expo Updates) provide escape hatches for critical JavaScript updates when app store review delays threaten business continuity, while Progressive Web Apps offer an alternative distribution channel bypassing stores entirely for suitable use cases.

**Key Takeaways:**
- iOS builds require macOS runners (GitHub-hosted or self-hosted) due to Xcode licensing; Android builds run on standard Linux containers
- Use Fastlane to abstract platform-specific CLI tools (xcodebuild, gradle, aapt) into reusable lanes for build, test, and deploy phases
- Never store signing certificates or keystores in repositories; use Fastlane Match for iOS and CI secret managers for Android keystores
- AAB (Android App Bundle) is mandatory for Play Store; IPA for App Store; automate versioning using CI build numbers injected at build time
- Implement UI testing on real devices via cloud device farms to catch platform-specific issues (camera permissions, memory constraints, OS version differences)
- Use OTA updates sparingly for JavaScript fixes only; native code changes always require full app store review cycles
- Maintain separate provisioning profiles for development (device testing) and distribution (TestFlight/Play Store Internal) to prevent signing mismatches

**Next Chapter Preview:**
Chapter 62: Edge Computing and CI/CD explores deployment to resource-constrained environments at the network edge—IoT devices, retail point-of-sale systems, and 5G mobile edge computing (MEC) nodes. We will examine lightweight Kubernetes distributions (K3s, MicroK8s) optimized for ARM architectures and limited bandwidth, GitOps strategies for fleet management of thousands of distributed devices, and A/B testing methodologies for edge deployments where rollback latency must be minimized. The chapter covers container optimization techniques (distroless images, multi-arch builds) for edge hardware, delta update mechanisms to reduce bandwidth consumption, and security considerations for devices operating outside traditional data center perimeters with intermittent connectivity.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='60. aiml_model_deployment.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='62. edge_computing_and_cicd.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
